In [1]:
import json
import ray
from ray.rllib.agents.registry import get_agent_class
from ray.tune import run_experiments
from ray.tune.registry import register_env
import numpy as np
from flow.networks.ring import RingNetwork, ADDITIONAL_NET_PARAMS
from flow.utils.registry import make_create_env
from flow.utils.rllib import FlowParamsEncoder
from flow.core.params import SumoParams, EnvParams, InitialConfig, NetParams
from flow.core.params import VehicleParams, SumoCarFollowingParams
from flow.controllers import RLController, IDMController, ContinuousRouter
from gym.spaces.box import Box
from gym.spaces import Tuple
from flow.core.params import InFlows
from flow.controllers import SimLaneChangeController
from flow.networks import Network
import os
from flow.core.params import SumoLaneChangeParams

from env_merge import myEnv

ADDITIONAL_ENV_PARAMS = {
    "max_accel": 1,
    "max_decel": 1,
}

# time horizon of a single rollout
HORIZON = 1000
# number of rollouts per training iteration
N_ROLLOUTS = 20
# number of parallel workers
N_CPUS = 2

vehicles = VehicleParams()
vehicles.add("rl",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             num_vehicles=0)
vehicles.add("human",
             acceleration_controller=(IDMController, {}),
             lane_change_controller=(SimLaneChangeController, {}),
             #routing_controller=(ContinuousRouter, {}),
             car_following_params=SumoCarFollowingParams(
                 speed_mode="obey_safe_speed",  
                 # we use the speed mode "obey_safe_speed" for better dynamics at the merge
             ),
             lane_change_params = SumoLaneChangeParams(lane_change_mode="strategic",lcpushy=1.0),
             num_vehicles=0)

# specify the edges vehicles can originate on
initial_config = InitialConfig(
    edges_distribution=["gneE6","gneE7"]
)
    
# specify the routes for vehicles in the network
class Network(Network):

    def specify_routes(self, net_params):
        return {
                "gneE7": ["gneE7","gneE6.243"],
                "gneE6": ["gneE6","gneE6.243"]
               }


inflow = InFlows()

#highway vehicles
inflow.add(veh_type="human",
           edge="gneE6",
           vehs_per_hour=5000,
            depart_lane="random",
            depart_speed="random",
            color="white")

#merging vehicles
inflow.add(veh_type="rl",
           edge="gneE7",
           vehs_per_hour=1000,
            depart_lane="random",
            depart_speed="random",
            color="blue")
inflow.add(veh_type="rl",
           edge="gneE7",
           vehs_per_hour=500,
            depart_lane="random",
            depart_speed="random",
            color="red")


fileDir='/mnt/c/Users/llave/Documents/GitHub/flow_osuphysics/lucalavezzo/merge.net.xml'
net_params = NetParams(
    template=fileDir,
    inflows=inflow
)

flow_params = dict(
    # name of the experiment
    exp_tag="merge",

    # name of the flow environment the experiment is running on
    env_name=myEnv,  # <------ here we replace the environment with our new environment

    # name of the network class the experiment is running on
    network=Network,

    # simulator that is used by the experiment
    simulator='traci',

    # sumo-related parameters (see flow.core.params.SumoParams)
    sim=SumoParams(
        sim_step=0.1,
        render=False,
        restart_instance=True,
    ),

    # environment related parameters (see flow.core.params.EnvParams)
    env=EnvParams(
        horizon=HORIZON,
        warmup_steps=750,
        clip_actions=False,
        additional_params={
            "sort_vehicles": False,
            "max_accel": 1,
            "max_decel": 1,
        },
    ),

    # network-related parameters (see flow.core.params.NetParams and the
    # network's documentation or ADDITIONAL_NET_PARAMS component)
    net=net_params,

    # vehicles to be placed in the network at the start of a rollout (see
    # flow.core.params.VehicleParams)
    veh=vehicles,

    # parameters specifying the positioning of vehicles upon initialization/
    # reset (see flow.core.params.InitialConfig)
    initial=initial_config
)


def setup_exps():
    """Return the relevant components of an RLlib experiment.

    Returns
    -------
    str
        name of the training algorithm
    str
        name of the gym environment to be trained
    dict
        training configuration parameters
    """
    alg_run = "PPO"

    agent_cls = get_agent_class(alg_run)
    config = agent_cls._default_config.copy()
    config["num_workers"] = N_CPUS
    config["train_batch_size"] = HORIZON * N_ROLLOUTS
    config["gamma"] = 0.999  # discount rate
    config["model"].update({"fcnet_hiddens": [3, 3]})
    config["use_gae"] = True
    config["lambda"] = 0.97
    config["kl_target"] = 0.02
    config["num_sgd_iter"] = 10
    config['clip_actions'] = False  # FIXME(ev) temporary ray bug
    config["horizon"] = HORIZON

    # save the flow params for replay
    flow_json = json.dumps(
        flow_params, cls=FlowParamsEncoder, sort_keys=True, indent=4)
    config['env_config']['flow_params'] = flow_json
    config['env_config']['run'] = alg_run

    create_env, gym_name = make_create_env(params=flow_params, version=0)

    # Register as rllib env
    register_env(gym_name, create_env)
    return alg_run, gym_name, config




/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [ ]:
alg_run, gym_name, config = setup_exps()
ray.init(num_cpus=N_CPUS + 1)
trials = run_experiments({
    flow_params["exp_tag"]: {
        "run": alg_run,
        "env": gym_name,
        "config": {
            **config
        },
        "checkpoint_freq": 20,
        "checkpoint_at_end": True,
        "max_failures": 999,
        "stop": {
            "training_iteration": 200,
        },
    }
})


2020-03-17 11:54:34,006	WARNING services.py:592 -- setpgrp failed, processes may not be cleaned up properly: [Errno 1] Operation not permitted.
2020-03-17 11:54:34,009	INFO resource_spec.py:212 -- Starting Ray with 5.62 GiB memory available for workers and up to 2.83 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-17 11:54:34,677	WARNING services.py:1080 -- Failed to start the dashboard. The dashboard requires Python 3 as well as 'pip install aiohttp psutil setproctitle grpcio'.


Trial name,status,loc
PPO_myEnv-v0_99c8a8de,RUNNING,


(pid=130) E0317 11:54:37.037067000     130 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584460477.037039700","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=130) E0317 11:54:37.037477000     130 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=129) E0317 11:54:36.926637100     129 socket_utils_common_posix.cc:208] check for SO_REUSEPORT: {"created":"@1584460476.926615400","description":"Protocol not available","errno":92,"file":"external/com_github_grpc_grpc/src/core/lib/iomgr/socket_utils_common_posix.cc","file_line":185,"os_error":"Protocol not available","syscall":"getsockopt(SO_REUSEPORT)"}
(pid=129) E0317 11:54:36.926917200     129 socket_utils_common_posix.cc:313] setsockopt(TCP_USER_TIMEOUT) Protocol not available
(pid=raylet) E03

(pid=131) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=131)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=131) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=131)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=131) /home/llavezzo/.local/lib/python3.6/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=131)   _np_qint16 = np.dtype([("qint16", np.int16, 1)])
(pid

(pid=131) Warning: Vehicle 'flow_00.32' performs emergency braking with decel=-10.45 wished=4.50 severity=1.32, time=65.60.
(pid=130) Warning: Vehicle 'flow_10.10' performs emergency braking with decel=-31.57 wished=4.50 severity=6.02, time=65.50.
(pid=131) Warning: Vehicle 'flow_20.5' performs emergency braking with decel=-99.41 wished=4.50 severity=21.09, time=69.30.
(pid=130) Warning: Vehicle 'flow_00.35' performs emergency braking with decel=-12.91 wished=4.50 severity=1.87, time=68.70.
(pid=130) Warning: Vehicle 'flow_10.12' performs emergency braking with decel=-12.55 wished=4.50 severity=1.79, time=75.20.
(pid=131) Warning: Vehicle 'flow_10.12' performs emergency braking with decel=-72.94 wished=4.50 severity=15.21, time=77.60.
(pid=130) Warning: Vehicle 'flow_00.45' performs emergency braking with decel=-13.48 wished=4.50 severity=1.99, time=78.60.
(pid=131) Warning: Vehicle 'flow_00.48' performs emergency braking with decel=-9.25 wished=4.50 severity=1.06, time=81.10.
(pid=130

...

(pid=130) Warning: Vehicle 'flow_00.38' performs emergency braking with decel=-24.73 wished=4.50 severity=4.49, time=74.30.
(pid=130) Warning: Vehicle 'flow_00.38' performs emergency braking with decel=-9.84 wished=4.50 severity=1.19, time=77.50.
(pid=130) Warning: Vehicle 'flow_20.6' performs emergency braking with decel=-19.50 wished=4.50 severity=3.33, time=79.40.
(pid=130) Warning: Vehicle 'flow_00.49' performs emergency braking with decel=-15.78 wished=4.50 severity=2.51, time=82.50.
(pid=131) Warning: Vehicle 'flow_10.18' performs emergency braking with decel=-77.64 wished=4.50 severity=16.25, time=108.80.
(pid=130) Warning: Vehicle 'flow_00.49' performs emergency braking with decel=-9.36 wished=4.50 severity=1.08, time=85.10.
(pid=130) Warning: Vehicle 'flow_00.49' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=87.10.
(pid=130) Warning: Vehicle 'flow_00.56' performs emergency braking with decel=-11.52 wished=4.50 severity=1.56, time=91.80.
(pid=131) 

(pid=131) Warning: Vehicle 'flow_10.8' performs emergency braking with decel=-102.85 wished=4.50 severity=21.86, time=60.80.
(pid=130) Warning: Vehicle 'flow_00.0' performs emergency braking with decel=-127.57 wished=4.50 severity=27.35, time=22.20.
(pid=131) Warning: Vehicle 'flow_00.31' performs emergency braking with decel=-12.63 wished=4.50 severity=1.81, time=64.30.
(pid=130) Warning: Vehicle 'flow_00.0' performs emergency braking with decel=-10.26 wished=4.50 severity=1.28, time=24.60.
(pid=130) Warning: Vehicle 'flow_00.0' performs emergency braking with decel=-14.39 wished=4.50 severity=2.20, time=26.40.
(pid=130) Warning: Vehicle 'flow_00.4' performs emergency braking with decel=-9.00 wished=4.50 severity=1.00, time=31.70.
(pid=130) Warning: Vehicle 'flow_00.8' performs emergency braking with decel=-11.94 wished=4.50 severity=1.65, time=39.10.
(pid=130) Warning: Vehicle 'flow_10.5' performs emergency braking with decel=-100.24 wished=4.50 severity=21.28, time=41.10.
(pid=130) 

Trial name,status,loc,iter,total time (s),timesteps,reward
PPO_myEnv-v0_99c8a8de,RUNNING,192.168.0.24:129,1,337.486,20000,8751.61


(pid=131) Warning: Vehicle 'flow_00.41' performs emergency braking with decel=-9.85 wished=4.50 severity=1.19, time=76.80.
(pid=130) Warning: Vehicle 'flow_00.47' performs emergency braking with decel=-43.23 wished=4.50 severity=8.61, time=78.80.
(pid=131) Warning: Vehicle 'flow_10.12' performs emergency braking with decel=-29.18 wished=4.50 severity=5.49, time=81.00.
(pid=130) Warning: Vehicle 'flow_10.14' performs emergency braking with decel=-18.31 wished=4.50 severity=3.07, time=82.40.
(pid=131) Warning: Vehicle 'flow_00.46' performs emergency braking with decel=-12.65 wished=4.50 severity=1.81, time=84.30.
(pid=131) Warning: Vehicle 'flow_00.46' performs emergency braking with decel=-11.07 wished=4.50 severity=1.46, time=88.60.
(pid=131) Warning: Vehicle 'flow_10.14' performs emergency braking with decel=-97.76 wished=4.50 severity=20.72, time=90.80.
(pid=131) Warning: Vehicle 'flow_20.7' performs emergency braking with decel=-10.71 wished=4.50 severity=1.38, time=90.90.
(pid=130)

(pid=131) Warning: Vehicle 'flow_00.11' performs emergency braking with decel=-10.93 wished=4.50 severity=1.43, time=36.40.
(pid=131) Warning: Vehicle 'flow_10.4' performs emergency braking with decel=-39.16 wished=4.50 severity=7.70, time=37.90.
(pid=130) Warning: Vehicle 'flow_10.1' performs emergency braking with decel=-102.24 wished=4.50 severity=21.72, time=28.00.
(pid=130) Warning: Vehicle 'flow_10.2' performs emergency braking with decel=-14.05 wished=4.50 severity=2.12, time=28.10.
(pid=130) Warning: Vehicle 'flow_10.1' performs emergency braking with decel=-19.80 wished=4.50 severity=3.40, time=30.20.
(pid=130) Warning: Vehicle 'flow_00.16' performs emergency braking with decel=-10.29 wished=4.50 severity=1.29, time=31.70.
(pid=131) Warning: Vehicle 'flow_00.14' performs emergency braking with decel=-9.30 wished=4.50 severity=1.07, time=46.00.
(pid=130) Warning: Vehicle 'flow_00.16' performs emergency braking with decel=-80.29 wished=4.50 severity=16.84, time=35.30.
(pid=130) 